# CNN model

In [5]:
# Dependencies
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import copy
import sys
import sklearn.metrics as metrics
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError
from torchinfo import summary
import parameters
import random
from data_formatting import split_sequence_overlap, split_sequence_nooverlap, split_sequence, split_train_test, normalize_data, set_targets
parameters.initialize_parameters()

## Architecture

In [6]:
"""
class CNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_length):
        super(CNN, self).__init__()
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.kernel_size1 = 7
        self.kernel_size2 = 7
        self.fc_input_size = hidden_size*(seq_length-self.kernel_size1+1-self.kernel_size2+1)
        self.conv1 = nn.Conv2d(1, hidden_size, (self.kernel_size1,1))   # output size : seq_length - 4
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, (self.kernel_size2,1))   # output size : seq_length - 8
        self.fc = nn.Linear(self.fc_input_size, 2)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
    
    def forward(self,x):
        x = torch.unsqueeze(x, 1)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1, self.fc_input_size) #reshaping the data for next dense layer
        x = self.fc(x)
        x[:,1] = self.sig(x[:,1])
        return x
"""
class CNN(nn.Module):
    def __init__(self, input_dim, seq_dim):
        super(CNN, self).__init__()
        
        # Define the architecture with layers based on the input arguments
        self.conv1 = nn.Conv1d(seq_dim, 16, 5)
        self.conv2 = nn.Conv1d(16, 32, 3)
        self.relu = nn.ReLU()
        
        self.fc_input_size = 32 * (input_dim - 5 - 3 + 2)
        self.fc = nn.Linear(self.fc_input_size, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Forward pass
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc(x)
        x = self.sigmoid(x)
        
        return x


## Parameters

In [7]:
# Classes we want to predict and binary outputs
list_targets = [0, 3]
list_labels = [0, 1]

# number of subjects used for validation
num_validation_subjects = 1

learning_rate = 0.0007
weight_decay = 10e-4
epochs = 3

print(torch.__version__)
device = torch.device('cpu')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")

2.0.1+cu117
Using cpu


In [8]:
# Get data
csvfile = "../../data/video/All_Subs_Diff_Modules_nofilter_withoutAUc.csv"
train_df = pd.read_csv(csvfile,  delimiter=",")  # 101 features (only AU_r)

# Select only the classes we want to predict
train_df, nclasses, targets_numpy = set_targets(train_df, list_targets, list_labels)

# Convert the subject names (strings) into numbers
subjects = pd.factorize(train_df['Subject'])[0]

# Normalise the features
features_numpy = normalize_data(train_df, False)# parameters.normalise_individual_subjects
input_dim = features_numpy.shape[1]
print(f"Number of features: {input_dim}")

del train_df

test_accuracies = []
calibrated_test_accuracies = []
all_predicted = []
all_labels = []
all_outputs = np.empty((0, nclasses), dtype='float')

# Get distinct subjects
subj = np.unique(subjects)

# Loop over all subjects
for test_subj in subj:
  xv_max_val = 0
  avg_test_acc = 0
  val_acc_val_loss_list = []
  test_acc_list = []

  # Cross validation
  for xv in range(parameters.cross_validation_passes):

    test_idx = np.array([test_subj])

    # Take out test subject from trainval (Crooss validation)
    trainval_idx = np.delete(subj, np.where(subj==test_subj))
    val_idx = trainval_idx[random.sample(range(len(trainval_idx)), num_validation_subjects)]
    val_idx = val_idx%len(subj)

    # Remove test & validation subjects from trainval
    train_idx = np.setxor1d(subj, test_idx)
    train_idx = np.setxor1d(train_idx, val_idx)

    print("Generating train/val/test split...")
    features_train, targets_train, features_val, targets_val, features_test, targets_test = split_train_test(targets_numpy, features_numpy, subjects, train_idx, val_idx, test_idx)

    print("Generating sequences...")
    features_train, targets_train = split_sequence_overlap(features_train, targets_train, parameters.seq_dim, parameters.overlap_size)
    features_val, targets_val = split_sequence_overlap(features_val, targets_val, parameters.seq_dim, parameters.overlap_size)
    
    # Overlap or no
    if parameters.test_with_subsequences:
      features_test, targets_test = split_sequence_overlap(features_test, targets_test, parameters.test_seq_dim, parameters.test_overlap_size)
    else:
      features_test, targets_test = split_sequence_nooverlap(features_test, targets_test, parameters.test_seq_dim, parameters.test_overlap_size)

    print(f"Number of training examples: {len(targets_train)}")
    print(f"Number of validation examples: {len(targets_val)}")
    print(f"Number of test examples: {len(targets_test)}")

    # Create feature and targets tensor for train set. We need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
    featuresTrain = torch.from_numpy(features_train)
    targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor)  # data type is long

    featuresVal = torch.from_numpy(features_val)
    targetsVal = torch.from_numpy(targets_val).type(torch.LongTensor)  # data type is long

    # Pytorch train and validation sets
    train = TensorDataset(featuresTrain, targetsTrain)
    val = TensorDataset(featuresVal, targetsVal)
    
    # Data loader
    train_loader = DataLoader(train, batch_size=parameters.batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=parameters.batch_size, shuffle=False)

    # Create feature and targets tensor for test set
    if parameters.test_with_subsequences:
      featuresTest = torch.from_numpy(features_test)
      targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor)  # data type is long
      test = TensorDataset(featuresTest, targetsTest)
      test_loader = DataLoader(test, batch_size=parameters.batch_size, shuffle=False)
    
    # Model
    model = CNN(input_dim, parameters.seq_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    cur_learning_rate = learning_rate
    error = nn.CrossEntropyLoss()
    error_cpu = nn.CrossEntropyLoss().to('cpu')

    # Train the model
    for epoch in range(epochs):
      model.train()
      running_loss = 0
      for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = error(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
      print(f"Epoch: {epoch+1}, Training Loss: {running_loss/len(train_loader)}")
    print("----Finished this cross validation pass----")



Number of features: 100
Generating train/val/test split...
Generating sequences...
Number of training examples: 1412
Number of validation examples: 70
Number of test examples: 672
Epoch: 1, Training Loss: 0.5193742945622862
Epoch: 2, Training Loss: 0.4254317009047176
Epoch: 3, Training Loss: 0.36765916595298254
----Finished this cross validation pass----
Generating train/val/test split...
Generating sequences...
Number of training examples: 1390
Number of validation examples: 86
Number of test examples: 672
Epoch: 1, Training Loss: 0.48754825434465515
Epoch: 2, Training Loss: 0.4011658827463786
Epoch: 3, Training Loss: 0.3653666932007362
----Finished this cross validation pass----
Generating train/val/test split...
Generating sequences...
Number of training examples: 1384
Number of validation examples: 84
Number of test examples: 672
Epoch: 1, Training Loss: 0.5224646621051876
Epoch: 2, Training Loss: 0.4232322470895175
Epoch: 3, Training Loss: 0.37816572668908655
----Finished this cro

In [9]:
# save the model
PATH = './model_test.pth'
torch.save(model.state_dict(), PATH)

In [10]:
for data, target in train_loader:
    print(data.shape)

torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000

torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([2, 1000, 100])


In [12]:
# Open the model file
model = CNN(input_dim, parameters.seq_dim).to(device)
model.load_state_dict(torch.load(PATH))


correct = 0
total = 0
prev_label = -1
class_hist = np.zeros(nclasses, dtype='int')


print(parameters.test_seq_dim)
# Iterate through test dataset
model.eval()
with torch.no_grad():
    if parameters.test_with_subsequences:
        for features, labels in test_loader:
            features = Variable(features.view(-1, parameters.test_seq_dim, input_dim)).to(device)
            labels = Variable(labels).to('cpu')

            print(features.shape)

            # Forward propagation
            outputs = model(features)

            test_loss = error_cpu(outputs.to('cpu'), labels)
            # Get predictions from the maximum value
            predicted = torch.max(outputs.data, 1)[1]
            predicted = predicted.to('cpu')

            # Total number of labels
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_predicted.extend(list(predicted.detach().numpy()))
            all_labels.extend(list(labels.detach().numpy()))
            all_outputs = np.concatenate((all_outputs, outputs.data.to('cpu').reshape(-1, nclasses)))

    
    else:
        count=0
        for features in features_test:
            features = torch.tensor(features)
            features = torch.unsqueeze(features, 0).to(device)
            labels = torch.unsqueeze(torch.tensor(targets_test[count]), 0)
            #features = Variable(features.view(-1, seq_dim, input_dim)).to(device)

            # Forward propagation
            outputs = model(features)

            test_loss = error(outputs.to('cpu'), labels)
            # Get predictions from the maximum value
            predicted = torch.max(outputs.data, 1)[1]
            predicted = predicted.to('cpu')

            # Total number of labels
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            count += 1

    al_np = np.array(all_labels)   
    ao_np = np.array(all_outputs)  

    accuracy = correct / float(total)

    print(f"Test accuracy for run {xv}: {accuracy}")

avg_test_acc += accuracy
test_acc_list.append(accuracy)

avg_test_acc = np.mean(test_acc_list)

# avg_test_acc /= parameters.cross_validation_passes
test_accuracies.append(avg_test_acc)
print("Test accuracies:")
print(test_accuracies)
print(f"Mean accuracy: {np.mean(test_accuracies)}")

1000
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([16, 1000, 100])
torch.Size([15,